# Q1

In [4]:
import string 
remove = dict.fromkeys(map(ord, string.punctuation))


#pre-processing the IMDB files 
with open("IMDB-Old-train.txt", "r", encoding="utf8")  as trainI:
    trainI =  trainI.read().translate(remove)
    trainI = trainI.lower()

with open("IMDB-Old-valid.txt", "r", encoding="utf8")  as validI:
    validI =  validI.read().translate(remove)
    validI = validI.lower()
    
with open("IMDB-Old-test.txt", "r", encoding="utf8")  as testI:
    testI =  testI.read().translate(remove)
    testI = testI.lower()

#pre-processing the Yelp files
with open("yelp-Old-train.txt", "r", encoding="utf8")  as trainY:
    trainY =  trainY.read().translate(remove)
    trainY = trainY.lower()

with open("yelp-Old-valid.txt", "r", encoding="utf8")  as validY:
    validY =  validY.read().translate(remove)
    validY = validY.lower()
    
with open("yelp-Old-test.txt", "r", encoding="utf8")  as testY:
    testY =  testY.read().translate(remove)
    testY = testY.lower()

In [5]:
from collections import Counter

with open("IMDB-vocab.txt", "w", encoding="utf8")  as vocab:
    vocabI = Counter(trainI.split()).most_common(10000)
    for i in range(10000):
        if (vocabI[i][0] != '1' and vocabI[i][0] !='0' and vocabI[i][0] != '\t'):
            vocab.write(vocabI[i][0] + '\t'+ str(i) +'\t'+ str(vocabI[i][1])+'\n')
    
with open("yelp-vocab.txt", "w", encoding="utf8")  as vocab:           
    vocabY = Counter(trainY.split()).most_common(11000)
    for i in range(10000):
        if (vocabY[i][0]!= '0' and vocabY[i][0] != '1' and vocabY[i][0]!='2' and vocabY[i][0] != '3' and vocabY[i][0] != '4' and vocabY[i][0] != '5' and vocabY[i][0]!='\t'):
            vocab.write(vocabY[i][0] + '\t'+ str(i)  +'\t'+ str(vocabY[i][1])+'\n')



In [16]:
def createFilesI(filePath, file):
    with open(filePath, "w", encoding="utf8")  as train:
        words = file.split()
        for word in words:
            if (word != '0' and word != '1'):
                for i in range(10000):
                    if word == vocabI[i][0]:
                        train.write(str(i) + ' ')
                        break
            else:
                train.write('\t' + word + '\n')

def createFilesY(filePath, file):
    with open(filePath, "w", encoding="utf8")  as train:
        words = file.split()
        for word in words:
            if (word != '0' and word != '1' and word !='2' and word != '3' and word != '4' and word != '5'):
                for i in range(10000):
                    if word == vocabY[i][0]:
                        train.write(str(i) + ' ')
                        break
            else:
                train.write('\t' + word + '\n')

                
    

In [42]:
#creating datasets

createFilesI("IMDB-train.txt", trainI)
createFilesI("IMDB-valid.txt", validI)
createFilesI("IMDB-test.txt", testI)

createFilesY("Yelp-train.txt", trainY)
createFilesY("Yelp-valid.txt", validY)
createFilesY("Yelp-test.txt", testY)


KeyboardInterrupt: 

In [17]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import normalize
import numpy as np
import pandas as pd

vocabulary = {}
vocabulary['IMDB'] = {word[0]: i for i, word in enumerate(vocabI)}
vocabulary['Yelp'] = {word[0]: i for i, word in enumerate(vocabY)}

IMDB_train = pd.read_csv("IMDB-Old-train.txt", sep = '\t', lineterminator ='\n', header=None, names = ['review', 'label'])
IMDB_valid = pd.read_csv("IMDB-Old-valid.txt", sep = '\t', lineterminator ='\n', header=None, names = ['review', 'label'])
IMDB_test = pd.read_csv("IMDB-Old-test.txt", sep = '\t', lineterminator ='\n', header=None, names = ['review', 'label'])
Yelp_train = pd.read_csv("Yelp-Old-train.txt", sep = '\t', lineterminator ='\n', header=None, names = ['review', 'label'])
Yelp_valid = pd.read_csv("Yelp-Old-valid.txt", sep = '\t', lineterminator ='\n', header=None, names = ['review', 'label'])
Yelp_test = pd.read_csv("IMDB-Old-test.txt", sep = '\t', lineterminator ='\n', header=None, names = ['review', 'label'])

datasets = {
    'IMDB':{'train': IMDB_train, 'valid': IMDB_valid, 'test':IMDB_test},
    'Yelp':{'train': Yelp_train, 'valid': Yelp_valid, 'test': Yelp_test}
}


def bow (dataset, vocabulary, xName='review', yName='label'):
    BBOW = {}
    FBOW = {}
    vectorizer = CountVectorizer(vocabulary = vocabulary)
    for splitName, split in dataset.items():
        vector = vectorizer.fit_transform(split[xName])   
        FBOW[splitName] = [normalize(vector.astype(np.float64)),split[yName] ]
        vector[vector > 1] = 1
        BBOW[splitName] = [vector, split[yName]]
    return BBOW, FBOW

IMDB_BBOW, IMDB_FBOW = bow (datasets['IMDB'], vocabulary['IMDB'])
Yelp_BBOW, Yelp_FBOW = bow(datasets['Yelp'], vocabulary['Yelp'])


# Q2.a

In [32]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import f1_score
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV


def calculate_base(train, test):
    print('F1-score for random and majority class classifier')
    dummy = DummyClassifier(strategy = 'uniform')
    predict = dummy.fit(train, test).predict(train)
    scoreDummy = f1_score(test, predict, average='macro')*100

    majority = DummyClassifier(strategy = 'most_frequent')
    predict = majority.fit(train, test).predict(train)
    scoreMajority = f1_score(test, predict, average='macro')*100

    print(scoreDummy, scoreMajority)

# Q2.b

In [79]:
#Tuning for  different hyperparameters 


def get_BernoulliNBParams(train, test, hyperparams):
    bern = BernoulliNB()
    clf = GridSearchCV(bern, hyperparams, cv=5, scoring='f1_micro', return_train_score=True).fit(train, test)
    print('The performance of hyperparameters in Bernoulli Naive Bayes: ')
    print(pd.DataFrame.from_dict(clf.cv_results_))
    print('\n')
    print('Best hyperparameter:')
    print(clf.best_params_)
    
        
def get_DecisionTreesParams(train, test, hyperparams):
    dtc = DecisionTreeClassifier()
    clf = GridSearchCV(dtc, hyperparams, cv=5, scoring='f1_micro', return_train_score=True).fit(train, test)
    print('The performance of hyperparameters in Decision Tree: ')
    print(pd.DataFrame.from_dict(clf.cv_results_))
    print('\n')
    print('Best hyperparameter:')
    print(clf.best_params_)
    
        
def get_SVCParams(train, test, hyperparams):
    svm = LinearSVC()
    clf = GridSearchCV(svm, hyperparams, cv=5, scoring='f1_micro', return_train_score=True).fit(train, test)
    print('The performance of hyperparameters in Linear SVC: ')
    print(pd.DataFrame.from_dict(clf.cv_results_))
    print('\n')
    print('Best hyperparameter:')
    print(clf.best_params_)
        

In [96]:
calculate_base(Yelp_BBOW['train'][0],Yelp_BBOW['train'][1])
print('\n')

parametersBN = {'alpha': [0.0000001, 0.00001, 0.0001, 0.001, 0.1, 1, 10]}
get_BernoulliNBParams(Yelp_BBOW['valid'][0], Yelp_BBOW['valid'][1],parametersBN)
print('\n')

parametersDTC = {'max_depth': [10,12,14,16,20], 'min_samples_split': [2,4,6,8,10], "max_features": [1,4, 6, 20, 40, 50, 80, 100]}
get_DecisionTreesParams(Yelp_BBOW['valid'][0],Yelp_BBOW['valid'][1], parametersDTC)
print('\n')

parametersSVC = {'tol': [0.00001, 0.0001, 0.01, 1, 10], 'C': [0.00001, 0.001, 0.01, 1, 2, 10]}
get_SVCParams(Yelp_BBOW['valid'][0],Yelp_BBOW['valid'][1], parametersSVC)

F1-score for random and majority class classifier
18.32766877332954 10.426700464723279




C:\Users\ramsha\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


The performance of hyperparameters in Bernoulli Naive Bayes: 
   mean_fit_time  std_fit_time  mean_score_time  std_score_time param_alpha  \
0       0.005800      0.001596         0.002598        0.000800       1e-07   
1       0.003998      0.000632         0.001602        0.000493       1e-05   
2       0.003598      0.000801         0.001603        0.000492      0.0001   
3       0.004396      0.001014         0.001602        0.000493       0.001   
4       0.005001      0.001547         0.002194        0.000747         0.1   
5       0.003999      0.000002         0.001199        0.000399           1   
6       0.003798      0.000399         0.001399        0.000490          10   

              params  split0_test_score  split1_test_score  split2_test_score  \
0   {'alpha': 1e-07}           0.391089              0.365              0.380   
1   {'alpha': 1e-05}           0.400990              0.385              0.375   
2  {'alpha': 0.0001}           0.405941              0.375    

The performance of hyperparameters in Linear SVC: 
    mean_fit_time  std_fit_time  mean_score_time  std_score_time param_C  \
0        0.012988      0.000006         0.001001    8.869684e-07   1e-05   
1        0.012614      0.000510         0.001001    4.101908e-07   1e-05   
2        0.011600      0.001356         0.000997    6.940561e-06   1e-05   
3        0.009002      0.002094         0.001000    6.322596e-04   1e-05   
4        0.007393      0.000800         0.001400    4.894695e-04   1e-05   
5        0.016390      0.000490         0.001000    7.136645e-07   0.001   
6        0.014396      0.000486         0.000996    7.366478e-06   0.001   
7        0.011800      0.000412         0.000396    4.856339e-04   0.001   
8        0.007988      0.000014         0.000804    4.022559e-04   0.001   
9        0.006200      0.000406         0.001001    7.599534e-07   0.001   
10       0.021017      0.000668         0.000800    4.001390e-04    0.01   
11       0.019389      0.000498      

# Q2 d

In [142]:
def calculate_BernoulliNB(train, test, hyperparameter):
    print('F1-score for bernoulli for hyperparameter: '+str(hyperparameter))
    bern = BernoulliNB(alpha = hyperparameter)
    predict = bern.fit(train, test).predict(train)
    score = f1_score(test, predict, average='micro')*100
    print(score)

def calculate_DecisionTrees(train, test, maxDepth, maxFeatures, minSamplesSplit):
    print('F1-score for Decision Trees for hyperparameter: '+str(maxDepth)+ " "  +  str(maxFeatures) + " "+ str(minSamplesSplit))
    tree = DecisionTreeClassifier(max_depth=maxDepth, max_features=maxFeatures, min_samples_split=minSamplesSplit)
    predict = tree.fit(train, test).predict(train) 
    score = f1_score(test, predict, average='micro')*100
    print(score)
    
def calculate_SVC(train, test, c, tolerance):
    print('F1-score for Linear SVC for hyperparameter: '+str(c) + " "+ str(tolerance))
    svc = LinearSVC(C=c, tol=tolerance)
    predict = svc.fit(train,test).predict(train)
    score = f1_score(test, predict, average='micro')*100
    print(score)

def calculate_GaussianNB(train, test):
    print('F1-score for Gaussian:')
    gNB = GaussianNB()
    predict = gNB.fit(train,test).predict(train)
    score = f1_score(test, predict, average='micro')*100
    print(score)

In [119]:
print('F1-scores for train set\n')
calculate_BernoulliNB(Yelp_BBOW['train'][0], Yelp_BBOW['train'][1], 0.1)
print('\n')
calculate_DecisionTrees(Yelp_BBOW['train'][0], Yelp_BBOW['train'][1], 12, 40, 4)
print('\n')
calculate_SVC(Yelp_BBOW['train'][0], Yelp_BBOW['train'][1], 0.01, 1e-05)
print('\n')
print('F1-scores for valid set\n')
calculate_BernoulliNB(Yelp_BBOW['valid'][0], Yelp_BBOW['valid'][1], 0.1)
print('\n')
calculate_DecisionTrees(Yelp_BBOW['valid'][0], Yelp_BBOW['valid'][1], 12, 40, 4)
print('\n')
calculate_SVC(Yelp_BBOW['valid'][0], Yelp_BBOW['valid'][1], 0.01, 1e-05)
print('\n')
print('F1-scores for test set\n')
calculate_BernoulliNB(Yelp_BBOW['test'][0], Yelp_BBOW['test'][1], 0.1 )
print('\n')
calculate_DecisionTrees(Yelp_BBOW['test'][0], Yelp_BBOW['test'][1], 12, 40, 4)
print('\n')
calculate_SVC(Yelp_BBOW['test'][0], Yelp_BBOW['test'][1], 0.01, 1e-05)
print('\n')

F1-scores for train set

F1-score for bernoulli for hyperparameter: 0.1
71.15714285714286


F1-score for Decision Trees for hyperparameter: 12 40 4
38.457142857142856


F1-score for Linear SVC for hyperparameter: 0.01 1e-05
84.78571428571429


F1-scores for valid set

F1-score for bernoulli for hyperparameter: 0.1
91.10000000000001


F1-score for Decision Trees for hyperparameter: 12 40 4
41.0


F1-score for Linear SVC for hyperparameter: 0.01 1e-05
94.89999999999999


F1-scores for test set

F1-score for bernoulli for hyperparameter: 0.1
86.56800000000001


F1-score for Decision Trees for hyperparameter: 12 40 4
65.164


F1-score for Linear SVC for hyperparameter: 0.01 1e-05
93.64




# Q3 a

In [138]:
parametersDTC = {'max_depth': [40, 60, 80, 90, 100], 'min_samples_split': [10, 20, 30, 40, 50, 60, 100], "max_features": [1, 4, 6, 20, 40, 50, 80, 100]}
get_DecisionTreesParams(Yelp_FBOW['valid'][0],Yelp_FBOW['valid'][1], parametersDTC)
print('\n')

parametersSVC = {'tol': [0.01, 0.01, 0.1, 1, 10], 'C': [0.00001, 0.001, 0.01, 1, 2, 10]}
get_SVCParams(Yelp_FBOW['valid'][0],Yelp_FBOW['valid'][1], parametersSVC)

The performance of hyperparameters in Decision Tree: 
     mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0         0.007194      0.002230         0.001602    4.919825e-04   
1         0.004801      0.000402         0.000999    6.325960e-07   
2         0.005197      0.000979         0.001003    6.902455e-06   
3         0.004194      0.000393         0.001199    3.999240e-04   
4         0.004801      0.000749         0.001405    4.964871e-04   
5         0.004590      0.000491         0.001196    4.013300e-04   
6         0.003798      0.000400         0.001598    4.973404e-04   
7         0.005993      0.000638         0.001599    4.822840e-04   
8         0.005592      0.000487         0.001000    1.168008e-07   
9         0.005204      0.000397         0.001397    4.848697e-04   
10        0.005400      0.000494         0.001200    4.000664e-04   
11        0.005396      0.000490         0.001000    7.893059e-07   
12        0.005593      0.000494         0.001196

The performance of hyperparameters in Linear SVC: 
    mean_fit_time  std_fit_time  mean_score_time  std_score_time param_C  \
0        0.008990      0.003031         0.001199    3.995944e-04   1e-05   
1        0.006796      0.000747         0.000800    4.002342e-04   1e-05   
2        0.004997      0.000633         0.000801    4.003290e-04   1e-05   
3        0.004798      0.000748         0.001000    8.529922e-07   1e-05   
4        0.006596      0.001356         0.001399    7.997279e-04   1e-05   
5        0.016590      0.006883         0.001999    2.095943e-03   0.001   
6        0.011993      0.002097         0.001002    4.062921e-06   0.001   
7        0.008403      0.000490         0.000800    4.001619e-04   0.001   
8        0.007596      0.002252         0.001995    2.529405e-03   0.001   
9        0.004898      0.000655         0.000800    4.002336e-04   0.001   
10       0.008598      0.000493         0.001000    1.038148e-06    0.01   
11       0.009002      0.000027      

# Q3.c

In [144]:
print('F1-scores for train set\n')
calculate_GaussianNB((Yelp_BBOW['train'][0]).toarray(), Yelp_BBOW['train'][1])
print('\n')
calculate_DecisionTrees(Yelp_FBOW['train'][0], Yelp_FBOW['train'][1], 100,100,50 )
print('\n')
calculate_SVC(Yelp_FBOW['train'][0], Yelp_FBOW['train'][1], 1, 0.1)
print('\n')
print('F1-scores for valid set\n')
calculate_GaussianNB((Yelp_BBOW['valid'][0]).toarray(), Yelp_BBOW['valid'][1])
print('\n')
print('F1-scores for valid set\n')
calculate_DecisionTrees(Yelp_FBOW['valid'][0], Yelp_FBOW['valid'][1], 100, 100, 50)
print('\n')
calculate_SVC(Yelp_FBOW['valid'][0], Yelp_FBOW['valid'][1], 1, 0.1)
print('\n')
print('F1-scores for test set\n')
calculate_GaussianNB((Yelp_FBOW['test'][0]).toarray(), Yelp_FBOW['test'][1] )
print('\n')
print('F1-scores for test set\n')
calculate_DecisionTrees(Yelp_FBOW['test'][0], Yelp_FBOW['test'][1], 100, 100, 50)
print('\n')
calculate_SVC(Yelp_FBOW['test'][0], Yelp_FBOW['test'][1],1, 0.1)
print('\n')


F1-scores for train set

F1-score for Gaussian:
66.42857142857143


F1-score for Decision Trees for hyperparameter: 100 100 50
63.67142857142857


F1-score for Linear SVC for hyperparameter: 1 0.1
82.05714285714286


F1-scores for valid set

F1-score for Gaussian:
93.5


F1-scores for valid set

F1-score for Decision Trees for hyperparameter: 100 100 50
67.30000000000001


F1-score for Linear SVC for hyperparameter: 1 0.1
96.0


F1-scores for test set

F1-score for Gaussian:
80.652


F1-scores for test set

F1-score for Decision Trees for hyperparameter: 100 100 50
86.64


F1-score for Linear SVC for hyperparameter: 1 0.1
91.816




# Q4 a

In [147]:
calculate_base(IMDB_BBOW['train'][0],IMDB_BBOW['train'][1])
print('\n')

parametersBN = {'alpha': [0.0000001, 0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 20, 30]}
get_BernoulliNBParams(IMDB_BBOW['valid'][0], IMDB_BBOW['valid'][1],parametersBN)
print('\n')

parametersDTC = {'max_depth': [10,12,14,16,20], 'min_samples_split': [2,4,6,8,10], "max_features": [1000, 2000, 3000, 4000]}
get_DecisionTreesParams(IMDB_BBOW['valid'][0],IMDB_BBOW['valid'][1], parametersDTC)
print('\n')

parametersSVC = {'tol': [0.00001, 0.0001, 0.01, 1, 10], 'C': [0.00001, 0.001, 0.01, 1, 2, 10]}
get_SVCParams(IMDB_BBOW['valid'][0],IMDB_BBOW['valid'][1], parametersSVC)

F1-score for random and majority class classifier
49.97268497933066 33.33333333333333




C:\Users\ramsha\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


The performance of hyperparameters in Bernoulli Naive Bayes: 
   mean_fit_time  std_fit_time  mean_score_time  std_score_time param_alpha  \
0       0.050985      0.029030         0.008052    6.411677e-04       1e-07   
1       0.039577      0.006339         0.008008    1.674781e-03       1e-05   
2       0.035376      0.002333         0.006001    6.773102e-06      0.0001   
3       0.033981      0.000625         0.005996    9.047349e-07       0.001   
4       0.039776      0.005561         0.007197    1.719786e-03        0.01   
5       0.036975      0.003689         0.007396    1.019786e-03         0.1   
6       0.038976      0.006599         0.007605    1.197520e-03           1   
7       0.034180      0.000399         0.005597    4.900408e-04          10   
8       0.033179      0.000399         0.006001    7.491960e-06          20   
9       0.034376      0.000798         0.006198    9.792588e-04          30   

              params  split0_test_score  split1_test_score  split2_t

The performance of hyperparameters in Linear SVC: 
    mean_fit_time  std_fit_time  mean_score_time  std_score_time param_C  \
0        0.095745      0.006334         0.002400    4.897658e-04   1e-05   
1        0.088953      0.003946         0.002799    3.993827e-04   1e-05   
2        0.084552      0.010027         0.003202    1.937087e-03   1e-05   
3        0.066763      0.006174         0.002998    1.094893e-03   1e-05   
4        0.059163      0.002716         0.002199    3.998547e-04   1e-05   
5        0.112536      0.009473         0.003004    1.095604e-03   0.001   
6        0.104540      0.014199         0.002598    4.895324e-04   0.001   
7        0.083952      0.000633         0.002802    4.018987e-04   0.001   
8        0.072558      0.003609         0.002400    4.895318e-04   0.001   
9        0.057166      0.000980         0.002399    4.896681e-04   0.001   
10       0.146512      0.002249         0.002799    3.994708e-04    0.01   
11       0.138120      0.001165      

In [148]:
print('F1-scores for train set\n')
calculate_BernoulliNB(IMDB_BBOW['train'][0], IMDB_BBOW['train'][1], 1)
print('\n')
calculate_DecisionTrees(IMDB_BBOW['train'][0], IMDB_BBOW['train'][1], 16, 4000, 8)
print('\n')
calculate_SVC(IMDB_BBOW['train'][0], IMDB_BBOW['train'][1], 0.01, 1)
print('\n')
print('F1-scores for valid set\n')
calculate_BernoulliNB(IMDB_BBOW['valid'][0], IMDB_BBOW['valid'][1], 1)
print('\n')
calculate_DecisionTrees(IMDB_BBOW['valid'][0], IMDB_BBOW['valid'][1], 16, 4000, 8)
print('\n')
calculate_SVC(IMDB_BBOW['valid'][0], IMDB_BBOW['valid'][1], 0.01, 1)
print('\n')
print('F1-scores for test set\n')
calculate_BernoulliNB(IMDB_BBOW['test'][0], IMDB_BBOW['test'][1], 1 )
print('\n')
calculate_DecisionTrees(IMDB_BBOW['test'][0], IMDB_BBOW['test'][1], 16, 4000, 8)
print('\n')
calculate_SVC(IMDB_BBOW['test'][0], IMDB_BBOW['test'][1], 0.01, 1)
print('\n')

F1-scores for train set

F1-score for bernoulli for hyperparameter: 1
86.94


F1-score for Decision Trees for hyperparameter: 16 4000 8
82.69999999999999


F1-score for Linear SVC for hyperparameter: 0.01 1
96.24666666666667


F1-scores for valid set

F1-score for bernoulli for hyperparameter: 1
88.71


F1-score for Decision Trees for hyperparameter: 16 4000 8
84.46000000000001


F1-score for Linear SVC for hyperparameter: 0.01 1
96.87


F1-scores for test set

F1-score for bernoulli for hyperparameter: 1
87.08


F1-score for Decision Trees for hyperparameter: 16 4000 8
80.728


F1-score for Linear SVC for hyperparameter: 0.01 1
95.348




# Q5 a

In [145]:
parametersDTC = {'max_depth': [60, 80, 100, 200, 400, 600, 1000], 'min_samples_split': [10, 20, 40, 60, 80, 90, 100], "max_features": [20, 40, 60, 100, 200, 600, 800, 1000]}
get_DecisionTreesParams(IMDB_FBOW['valid'][0],IMDB_FBOW['valid'][1], parametersDTC)
print('\n')

parametersSVC = {'tol': [0.000001, 0.0001, 0.001, 0.1, 0.1 , 10], 'C': [0.000001, 0.0001, 0.01, 0.1, 1, 2, 10]}
get_SVCParams(IMDB_FBOW['valid'][0],IMDB_FBOW['valid'][1], parametersSVC)

The performance of hyperparameters in Decision Tree: 
     mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0         0.121332      0.041835         0.015013        0.007473   
1         0.110941      0.025621         0.010795        0.001832   
2         0.087150      0.002306         0.008999        0.000007   
3         0.081752      0.003485         0.009400        0.000487   
4         0.084755      0.010159         0.009392        0.001349   
5         0.082555      0.007831         0.009199        0.000978   
6         0.078555      0.002331         0.008214        0.000399   
7         0.134927      0.014549         0.008995        0.001092   
8         0.125728      0.009615         0.008799        0.000743   
9         0.106136      0.002317         0.008395        0.000490   
10        0.107934      0.010694         0.008596        0.000800   
11        0.110531      0.014221         0.009995        0.001788   
12        0.101740      0.015109         0.009000

The performance of hyperparameters in Linear SVC: 
    mean_fit_time  std_fit_time  mean_score_time  std_score_time param_C  \
0        0.047377      0.003822         0.001800    4.003796e-04   1e-06   
1        0.038183      0.001162         0.001999    2.429531e-06   1e-06   
2        0.032196      0.004117         0.001397    4.898255e-04   1e-06   
3        0.029592      0.001349         0.001402    4.853744e-04   1e-06   
4        0.029990      0.001409         0.001396    4.846703e-04   1e-06   
5        0.030591      0.001851         0.001994    7.583660e-06   1e-06   
6        0.072560      0.003718         0.002603    1.197665e-03  0.0001   
7        0.058968      0.001671         0.002398    4.899014e-04  0.0001   
8        0.052775      0.000979         0.001199    4.000682e-04  0.0001   
9        0.030989      0.002611         0.001601    4.877621e-04  0.0001   
10       0.030380      0.002578         0.001999    5.560829e-07  0.0001   
11       0.027785      0.000400      

In [146]:
print('F1-scores for train set\n')
calculate_GaussianNB((IMDB_FBOW['train'][0]).toarray(), IMDB_BBOW['train'][1])
print('\n')
calculate_DecisionTrees(Yelp_FBOW['train'][0], Yelp_FBOW['train'][1], 200, 800, 90)
print('\n')
calculate_SVC(IMDB_FBOW['train'][0], IMDB_FBOW['train'][1], 2, 1e-06)
print('\n')
print('F1-scores for valid set\n')
calculate_GaussianNB((IMDB_FBOW['valid'][0]).toarray(), IMDB_FBOW['valid'][1])
print('\n')
calculate_DecisionTrees(IMDB_FBOW['valid'][0], IMDB_FBOW['valid'][1], 200, 800, 90)
print('\n')
calculate_SVC(IMDB_FBOW['valid'][0], IMDB_FBOW['valid'][1], 2, 1e-06)
print('\n')
print('F1-scores for test set\n')
calculate_GaussianNB((IMDB_FBOW['test'][0]).toarray(), IMDB_FBOW['test'][1])
print('\n')
calculate_DecisionTrees(IMDB_FBOW['test'][0], IMDB_FBOW['test'][1], 200, 800, 90)
print('\n')
calculate_SVC(IMDB_FBOW['test'][0], IMDB_FBOW['test'][1], 2, 1e-06)
print('\n')

F1-scores for train set

F1-score for Gaussian:
86.30666666666667


F1-score for Decision Trees for hyperparameter: 200 800 90
62.87142857142857


F1-score for Linear SVC for hyperparameter: 2 1e-06
95.10666666666665


F1-scores for valid set

F1-score for Gaussian:
87.39


F1-score for Decision Trees for hyperparameter: 200 800 90
86.61


F1-score for Linear SVC for hyperparameter: 2 1e-06
95.36


F1-scores for test set

F1-score for Gaussian:
84.776


F1-score for Decision Trees for hyperparameter: 200 800 90
88.02799999999999


F1-score for Linear SVC for hyperparameter: 2 1e-06
94.5


